In [1]:
import math
import datetime
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import svm
from sklearn.linear_model  import LogisticRegression
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D,AveragePooling1D ,Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate,GRU
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [2]:
data= pd.read_csv('../Data/Final_direction.csv')
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [3]:
#data= pd.read_csv('./Dataset/f_working.csv')
#data.sort_values(by="Date",inplace=True)
#data.drop(['Unnamed: 0', 'Phase','Date'], axis=1, inplace=True)
#data["Target"] = (data["Price"].pct_change().shift(-1) > 0).astype(int)
#data.reset_index(drop=True,inplace=True)
#data.to_csv('./Dataset/Final_direction.csv')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
data

,Price,Open,High,Low,Vol.,Change %,AVBLS,CPTRA,CPTRV,DIFF,...,NADDU,NTRAN,NTRBL,TOUTV,TRFEE,neg_count,tot_count,iscore,PhaseNo,Target
0,32193.3,29359.7,33233.5,29008.0,240870,9.65,1.395637,107.041738,0.518649,1.859959e+13,...,711719.0,297111.0,1967.622517,2.164453e+06,79.699067,2055.082474,19557.546392,1.000000,0,1
1,32958.9,32192.9,34755.9,32029.6,155210,2.38,1.288694,100.029960,0.614294,1.859959e+13,...,812749.0,359116.0,2272.886076,2.211805e+06,86.419994,2055.082474,19557.546392,1.000000,0,0
2,32022.6,33016.6,33587.5,28204.5,255270,-2.84,1.318806,103.580664,0.529815,1.859959e+13,...,851667.0,373734.0,2160.312139,1.922522e+06,108.705036,2055.082474,19557.546392,1.000000,0,1
3,33991.5,32015.4,34414.7,30010.5,202130,6.15,1.292713,99.733618,0.521220,1.859959e+13,...,845343.0,354091.0,2255.356688,2.209914e+06,113.365867,2055.082474,19557.546392,1.000000,0,1
4,36793.2,33999.3,36934.8,33408.3,227560,8.24,1.293668,108.219166,0.569599,1.859959e+13,...,924851.0,397384.0,2245.107345,2.131443e+06,128.518985,2055.082474,19557.546392,1.000000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,50678.2,50783.6,52016.3,50459.0,43900,-0.20,1.228930,190.983398,1.185242,2.427233e+13,...,674082.0,249548.0,1686.135135,3.132094e+06,13.159126,2055.082474,19557.546392,1.000000,0,0
350,47545.2,50703.4,50703.8,47345.7,74390,-6.18,1.470745,163.363249,0.932660,2.427233e+13,...,729388.0,265422.0,1980.761194,3.275691e+06,17.860682,2055.082474,19557.546392,1.000000,0,0
351,46461.7,47548.4,48121.7,46127.8,63920,-2.28,1.402008,159.081648,0.702449,2.427233e+13,...,741024.0,276083.0,1904.020690,3.496397e+06,16.409159,614.000000,3770.000000,0.298771,0,1
352,47123.3,46470.7,47901.4,46003.0,60960,1.42,1.440240,144.215806,0.786106,2.427233e+13,...,679386.0,245021.0,2059.000000,3.153504e+06,17.654039,2813.000000,25088.000000,1.368802,0,0


In [5]:
data.corr()

,Price,Open,High,Low,Vol.,Change %,AVBLS,CPTRA,CPTRV,DIFF,...,NADDU,NTRAN,NTRBL,TOUTV,TRFEE,neg_count,tot_count,iscore,PhaseNo,Target
Price,1.000000,0.980573,0.991579,0.990524,-0.337493,0.040932,0.074218,0.766544,0.104644,0.395724,...,0.337275,0.245704,0.062399,0.224960,-0.002668,0.024130,0.048338,0.024130,-0.005476,-0.096625
Open,0.980573,1.000000,0.992438,0.985028,-0.313442,-0.151530,0.085614,0.768094,0.089948,0.413342,...,0.334305,0.235252,0.062087,0.232169,-0.000160,0.025208,0.044125,0.025208,0.012961,-0.078561
High,0.991579,0.992438,1.000000,0.985481,-0.283244,-0.055898,0.103757,0.755861,0.065695,0.424344,...,0.364008,0.262571,0.088149,0.229331,0.021368,0.027823,0.048505,0.027823,0.009372,-0.090565
Low,0.990524,0.985028,0.985481,1.000000,-0.409841,-0.031088,0.055717,0.778315,0.138317,0.375629,...,0.303534,0.217849,0.034659,0.226871,-0.040121,0.030655,0.054284,0.030655,-0.000868,-0.090562
Vol.,-0.337493,-0.313442,-0.283244,-0.409841,1.000000,-0.076501,0.337834,-0.425452,-0.490045,0.217716,...,0.341458,0.259984,0.400646,-0.160753,0.508388,-0.129287,-0.147894,-0.129287,0.068417,0.027109
Change %,0.040932,-0.151530,-0.055898,-0.031088,-0.076501,1.000000,-0.048113,-0.054239,0.046971,-0.109970,...,0.011737,0.052016,0.006658,-0.046162,-0.001834,0.004923,0.022748,0.004923,-0.093455,-0.077182
AVBLS,0.074218,0.085614,0.103757,0.055717,0.337834,-0.048113,1.000000,-0.337812,-0.606094,0.512672,...,0.548495,0.455654,0.702413,-0.159499,0.481074,-0.139500,-0.161788,-0.139500,-0.056408,-0.025291
CPTRA,0.766544,0.768094,0.755861,0.778315,-0.425452,-0.054239,-0.337812,1.000000,0.522231,0.125841,...,-0.116251,-0.203340,-0.514002,0.086316,-0.147869,0.002410,0.026637,0.002410,0.015166,-0.074961
CPTRV,0.104644,0.089948,0.065695,0.138317,-0.490045,0.046971,-0.606094,0.522231,1.000000,-0.219244,...,-0.546983,-0.492222,-0.681925,-0.055360,-0.365505,0.036099,0.047473,0.036099,-0.007561,-0.007948
DIFF,0.395724,0.413342,0.424344,0.375629,0.217716,-0.109970,0.512672,0.125841,-0.219244,1.000000,...,0.449374,0.264456,0.441454,-0.022385,0.393763,-0.250199,-0.262590,-0.250199,0.053689,-0.065981


In [5]:
# split dataset 80% train, 10% validation, 10% dev
def split_dataset(dataset):
    size = dataset.shape[0]
    train_size = size * 80 // 100
    test_size = size * 10 // 100
    
    return dataset[0:train_size, :], dataset[train_size:(train_size + test_size), :], dataset[(train_size + test_size):size, :]


In [6]:
data['y'] = data['Target']
data.drop(['Target'], axis=1, inplace=True)
dataset = data['y']
dataset.loc[dataset.shape[0]]= 0.0
dataset = dataset.iloc[1:]
data.iloc[:, -1] = dataset.values
dataset = data[:-1]
dataset_to_numpy = dataset.values


In [7]:
train_dataset, cross_validation_dataset, dev_dataset = split_dataset(dataset_to_numpy)

print("Dataset shape: {:s}".format(str(dataset_to_numpy.shape)))
print("Train dataset shape: {:s}".format(str(train_dataset.shape)))
print("Cross validation dataset shape: {:s}".format(str(cross_validation_dataset.shape)))
print("Dev dataset shape: {:s}".format(str(dev_dataset.shape)))

Dataset shape: (353, 26)
Train dataset shape: (282, 26)
Cross validation dataset shape: (35, 26)
Dev dataset shape: (36, 26)


In [8]:
def tf_dataset(series_x, series_y, batch_size, shuffle_buffer, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((series_x, series_y))
    if shuffle:
        ds = ds.cache().shuffle(shuffle_buffer).batch(batch_size).repeat()
    else:
        ds = ds.cache().batch(batch_size).repeat()

    return ds

def create_window_dataset(ds, lb, window_size):
    windowed_dataset = []
    labels = []
    for i in range(window_size, ds.shape[0] + 1):
        windowed_dataset.append(ds[i - window_size:i])
        labels.append(lb[i - 1])
    
    return np.array(windowed_dataset), np.array(labels)


In [9]:
scaler = MinMaxScaler()
scaler = scaler.fit(train_dataset[:, 0:-1])
train_dataset_normalized = scaler.transform(train_dataset[:, 0:-1])
cross_validation_dataset_normalized = scaler.transform(cross_validation_dataset[:, 0:-1])
dev_dataset_normalized = scaler.transform(dev_dataset[:, 0:-1])

In [10]:
WINDOW_SIZE = 14
BATCH_SIZE = 32

windowed_dataset_train, labels_train = create_window_dataset(train_dataset[:, 0:-1], train_dataset[:, -1], WINDOW_SIZE)

windowed_dataset_train, labels_train = create_window_dataset(train_dataset_normalized, train_dataset[:, -1], WINDOW_SIZE)
train_set = tf_dataset(windowed_dataset_train, labels_train, BATCH_SIZE, 1000)
unshuffled_train_set = tf_dataset(windowed_dataset_train, labels_train, BATCH_SIZE, 1000, False)

windowed_dataset_validation, labels_validation = create_window_dataset(cross_validation_dataset_normalized, cross_validation_dataset[:, -1], WINDOW_SIZE)
cross_validation_set = tf_dataset(windowed_dataset_validation, labels_validation, BATCH_SIZE, 1000, False)

windowed_dataset_dev, labels_dev = create_window_dataset(dev_dataset_normalized, dev_dataset[:, -1], WINDOW_SIZE)
dev_set = tf_dataset(windowed_dataset_dev, labels_dev, 1, 1000, False)

2022-06-06 11:34:32.667887: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
def n_dataset(ds):
    nparr=[]
    for i in ds:
        nparr.append(i[0])
    return np.array(nparr)

In [12]:
nt=n_dataset(windowed_dataset_train)
nv=n_dataset(windowed_dataset_validation)
nd=n_dataset(windowed_dataset_dev)
nd.shape

(23, 25)

In [13]:
TRAIN_STEP = math.ceil(windowed_dataset_train.shape[0] / BATCH_SIZE)
VALIDATION_STEP = math.ceil(windowed_dataset_validation.shape[0] / BATCH_SIZE)
DEV_STEP = windowed_dataset_dev.shape[0]

In [14]:
import time



In [39]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from numpy import mean
from numpy import std
lr=LogisticRegression()
X=np.concatenate((nt,nv))
y=np.concatenate((labels_train,labels_validation))
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scoring = ['precision_macro', 'recall_macro','accuracy']
scores = cross_validate(lr, X, y, scoring=scoring,cv=cv)
print('Precision: %.3f (%.3f)' % (mean(scores['test_precision_macro']), std(scores['test_precision_macro'])))
print('Recall: %.3f (%.3f)' % (mean(scores['test_recall_macro']), std(scores['test_recall_macro'])))
print('Accuracy: %.3f (%.3f)' % (mean(scores['test_accuracy']), std(scores['test_accuracy'])))

Precision: 0.519 (0.099)
Recall: 0.514 (0.102)
Accuracy: 0.509 (0.096)


In [40]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

svc=svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X,y)

scores = cross_validate(clf, X, y, scoring=scoring,cv=cv)
print('Precision: %.3f (%.3f)' % (mean(scores['test_precision_macro']), std(scores['test_precision_macro'])))
print('Recall: %.3f (%.3f)' % (mean(scores['test_recall_macro']), std(scores['test_recall_macro'])))
print('Accuracy: %.3f (%.3f)' % (mean(scores['test_accuracy']), std(scores['test_accuracy'])))

Precision: 0.527 (0.079)
Recall: 0.521 (0.075)
Accuracy: 0.519 (0.069)


In [15]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [58]:
trials=0
min_acc=0.56
val_acc=[]
dev_prec=[]

while trials<10:
    
    trials+=1
    tf.keras.backend.clear_session()

    cnn_p = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64,
                           kernel_size=9,
                           strides=1,
                           padding="same",
                           activation="tanh",
                           input_shape=windowed_dataset_train.shape[-2:]),
    tf.keras.layers.MaxPooling1D(pool_size=4, strides=1, padding="valid"),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="same", activation="tanh"),
    tf.keras.layers.MaxPooling1D(pool_size=2, strides=1, padding="valid"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation="tanh"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation="tanh"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    

    cnn_p.compile(loss='binary_crossentropy',
              optimizer='adam',
               metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    log_dir = "logs3\\" + datetime.datetime.now().strftime("%d-%m-%Y_%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    history = cnn_p.fit(train_set,
                    epochs=100,
                    steps_per_epoch=TRAIN_STEP,
                    validation_data=cross_validation_set,
                    validation_steps=VALIDATION_STEP,
                    verbose=0,
                    callbacks=[tensorboard_callback])
    forecast_train = cnn_p.predict(unshuffled_train_set, steps=TRAIN_STEP)
    forecast_validation = cnn_p.predict(cross_validation_set, steps=VALIDATION_STEP)
    forecast_dev = cnn_p.predict(dev_set, steps=DEV_STEP)
    total_forecast = np.concatenate((forecast_train[:,0], forecast_validation[:,0], forecast_dev[:,0]))
    total_labels = np.concatenate((labels_train, labels_validation, labels_dev))
    b=tf.keras.metrics.BinaryAccuracy(
        name="binary_accuracy", dtype=None,threshold=0.5
    )
    
    p=tf.keras.metrics.Precision(
        name=None, dtype=None,thresholds=0.5
    )
    
    r=tf.keras.metrics.Recall(
        name=None, dtype=None,thresholds=0.5
    )
    
    
#     acc_labels_dev= (forecast_dev < 0.5).astype('int64')

#     precision = precision_score(labels_dev, acc_labels_dev)
#     recall = recall_score(labels_dev, acc_labels_dev)
#     f1 = f1_score(labels_dev, acc_labels_dev)

#     print('Dev set precision:'+str(precision_p))
#     print(acc_labels_dev)
#     dev_prec.append((precision, recall, f1, acc))
    
    acc=b(labels_dev,forecast_dev).numpy()
    prec=p(labels_dev,forecast_dev).numpy()
    rec=r(labels_dev,forecast_dev).numpy()


    
    
    
    print(trials)
    print('Dev set accuracy:'+str(acc))
    #if acc>min_acc:
    #    ts = time.time()
    #    model.save('model/model_'+str(acc)+str(trials)+'-801010'+str(ts)+'_20.ckpt')
        
    print(forecast_dev)
    print('Validation accuracy: '+str(b(labels_validation,forecast_validation).numpy()))
    dev_prec.append((b(labels_validation,forecast_validation).numpy(),acc, prec, rec))
# sorted(val_acc,key=lambda x:x[0],reverse=True)

sorted(dev_prec,key=lambda x:x[0],reverse=True)

1
Dev set accuracy:0.5652174
[[0.00301647]
 [0.833428  ]
 [0.00506619]
 [0.00267351]
 [0.00299409]
 [0.12841544]
 [0.00265762]
 [0.8252365 ]
 [0.44844103]
 [0.99727464]
 [0.9969305 ]
 [0.04134652]
 [0.997427  ]
 [0.00285411]
 [0.9967387 ]
 [0.9952092 ]
 [0.97910655]
 [0.944377  ]
 [0.0029718 ]
 [0.00996843]
 [0.98269725]
 [0.67760813]
 [0.9974338 ]]
Validation accuracy: 0.5326087
2
Dev set accuracy:0.47826087
[[0.00394705]
 [0.72433305]
 [0.17493376]
 [0.00129378]
 [0.00103533]
 [0.00154209]
 [0.00101906]
 [0.9959327 ]
 [0.9656267 ]
 [0.9991038 ]
 [0.99889076]
 [0.99562883]
 [0.99887526]
 [0.00100848]
 [0.9872249 ]
 [0.9860905 ]
 [0.91722727]
 [0.9987496 ]
 [0.05512717]
 [0.94783795]
 [0.9419079 ]
 [0.09176758]
 [0.9971362 ]]
Validation accuracy: 0.42094862
3
Dev set accuracy:0.4347826
[[0.66445816]
 [0.98280156]
 [0.017037  ]
 [0.00537387]
 [0.00504944]
 [0.00898257]
 [0.00501806]
 [0.9865963 ]
 [0.9947807 ]
 [0.99533576]
 [0.9952263 ]
 [0.9911921 ]
 [0.99525565]
 [0.00504923]
 [0.066

[(0.5230567, 0.47826087, 0.42857143, 0.6),
 (0.5217391, 0.5652174, 0.5, 0.6),
 (0.49275362, 0.47826087, 0.4, 0.4),
 (0.47826087, 0.4347826, 0.36363637, 0.4),
 (0.44795784, 0.4347826, 0.4, 0.6),
 (0.43214753, 0.47826087, 0.4, 0.4),
 (0.43214753, 0.47826087, 0.42857143, 0.6),
 (0.40184453, 0.47826087, 0.42857143, 0.6),
 (0.38735178, 0.4347826, 0.4, 0.6),
 (0.3715415, 0.47826087, 0.42857143, 0.6)]

In [59]:
trials=0
min_acc=0.56
val_acc=[]
dev_metrics=[]
while trials<10:
    
    trials+=1
    tf.keras.backend.clear_session()
    model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=windowed_dataset_train.shape[-2:],return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation="tanh"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation="tanh"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    

    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    log_dir = "logs3\\" + datetime.datetime.now().strftime("%d-%m-%Y_%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    history = model.fit(train_set,
                    epochs=100,
                    steps_per_epoch=TRAIN_STEP,
                    validation_data=cross_validation_set,
                    validation_steps=VALIDATION_STEP,
                    verbose=0,
                    callbacks=[tensorboard_callback])
    forecast_train = model.predict(unshuffled_train_set, steps=TRAIN_STEP)
    forecast_validation = model.predict(cross_validation_set, steps=VALIDATION_STEP)
    forecast_dev = model.predict(dev_set, steps=DEV_STEP)
    total_forecast = np.concatenate((forecast_train[:,0], forecast_validation[:,0], forecast_dev[:,0]))
    total_labels = np.concatenate((labels_train, labels_validation, labels_dev))
   
    b=tf.keras.metrics.BinaryAccuracy(
        name="binary_accuracy", dtype=None,threshold=0.5
    )
    
    p=tf.keras.metrics.Precision(
        name=None, dtype=None,thresholds=0.5
    )
    
    r=tf.keras.metrics.Recall(
        name=None, dtype=None,thresholds=0.5
    )
    
    
#     acc_labels_dev= (forecast_dev < 0.5).astype('int64')

#     precision = precision_score(labels_dev, acc_labels_dev)
#     recall = recall_score(labels_dev, acc_labels_dev)
#     f1 = f1_score(labels_dev, acc_labels_dev)

# #     print('Dev set precision:'+str(precision_p))
# #     print(acc_labels_dev)
#     dev_metrics.append((precision, recall, f1, acc))
    
    acc=b(labels_dev,forecast_dev).numpy()
    prec=p(labels_dev,forecast_dev).numpy()
    rec=r(labels_dev,forecast_dev).numpy()



    print(trials)
    print('Dev set accuracy:'+str(acc))
    '''if acc>min_acc:
        ts = time.time()
        model.save('model/model_'+str(acc)+str(trials)+'-801010'+str(ts)+'_20.ckpt')
    '''
    print(forecast_dev)
    print('Validation accuracy: '+str(b(labels_validation,forecast_validation).numpy()))
    dev_metrics.append((b(labels_validation,forecast_validation).numpy(),acc, prec, rec))

sorted(dev_metrics,key=lambda x:x[0],reverse=True)



1
Dev set accuracy:0.39130434
[[0.9127408 ]
 [0.9068322 ]
 [0.8693824 ]
 [0.36174852]
 [0.13724816]
 [0.37033278]
 [0.52352977]
 [0.8007271 ]
 [0.88387775]
 [0.8988921 ]
 [0.8848531 ]
 [0.7840834 ]
 [0.5673841 ]
 [0.373684  ]
 [0.40612525]
 [0.6548547 ]
 [0.6198017 ]
 [0.7068012 ]
 [0.6799155 ]
 [0.7619575 ]
 [0.80604863]
 [0.7324592 ]
 [0.79145765]]
Validation accuracy: 0.4229249
2
Dev set accuracy:0.39130434
[[0.7190164 ]
 [0.6007515 ]
 [0.57719094]
 [0.3212928 ]
 [0.1747492 ]
 [0.25158513]
 [0.40337387]
 [0.7010853 ]
 [0.80656767]
 [0.8397924 ]
 [0.82818377]
 [0.7455107 ]
 [0.63904846]
 [0.5029069 ]
 [0.4850314 ]
 [0.6252361 ]
 [0.6378778 ]
 [0.7072743 ]
 [0.675879  ]
 [0.676731  ]
 [0.7311893 ]
 [0.6795322 ]
 [0.72461176]]
Validation accuracy: 0.4456522
3
Dev set accuracy:0.3478261
[[0.7933417 ]
 [0.7459511 ]
 [0.68736315]
 [0.23340032]
 [0.14850014]
 [0.37526298]
 [0.54526067]
 [0.78457236]
 [0.8595877 ]
 [0.8702711 ]
 [0.8434799 ]
 [0.6682915 ]
 [0.432963  ]
 [0.2692722 ]
 [0.244

[(0.46508566, 0.3043478, 0.3125, 0.5),
 (0.46376812, 0.39130434, 0.3888889, 0.7),
 (0.44927537, 0.3478261, 0.3529412, 0.6),
 (0.4347826, 0.3043478, 0.2857143, 0.4),
 (0.4334651, 0.39130434, 0.3888889, 0.7),
 (0.40316203, 0.39130434, 0.35714287, 0.5),
 (0.38866928, 0.3478261, 0.33333334, 0.5),
 (0.38866928, 0.3478261, 0.3529412, 0.6),
 (0.38735178, 0.4347826, 0.42105263, 0.8),
 (0.35836628, 0.3478261, 0.3529412, 0.6)]

In [65]:
trials=0
min_acc=0.56
val_acc=[]
dev_metrics=[]
while trials<10:
    
    trials+=1
    tf.keras.backend.clear_session()
    model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(64, input_shape=windowed_dataset_train.shape[-2:],return_sequences=True),
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation="tanh"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation="tanh"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    

    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    log_dir = "logs3\\" + datetime.datetime.now().strftime("%d-%m-%Y_%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    history = model.fit(train_set,
                    epochs=100,
                    steps_per_epoch=TRAIN_STEP,
                    validation_data=cross_validation_set,
                    validation_steps=VALIDATION_STEP,
                    verbose=0,
                    callbacks=[tensorboard_callback])
    forecast_train = model.predict(unshuffled_train_set, steps=TRAIN_STEP)
    forecast_validation = model.predict(cross_validation_set, steps=VALIDATION_STEP)
    forecast_dev = model.predict(dev_set, steps=DEV_STEP)
    total_forecast = np.concatenate((forecast_train[:,0], forecast_validation[:,0], forecast_dev[:,0]))
    total_labels = np.concatenate((labels_train, labels_validation, labels_dev))
    b=tf.keras.metrics.BinaryAccuracy(
        name="binary_accuracy", dtype=None,threshold=0.5
    )
    
    p=tf.keras.metrics.Precision(
        name=None, dtype=None,thresholds=0.5
    )
    
    r=tf.keras.metrics.Recall(
        name=None, dtype=None,thresholds=0.5
    )
    
    
#     acc_labels_dev= (forecast_dev < 0.5).astype('int64')

#     precision = precision_score(labels_dev, acc_labels_dev)
#     recall = recall_score(labels_dev, acc_labels_dev)
#     f1 = f1_score(labels_dev, acc_labels_dev)
#     dev_metrics.append((precision, recall, f1, acc))
    
    acc=b(labels_dev,forecast_dev).numpy()
    prec=p(labels_dev,forecast_dev).numpy()
    rec=r(labels_dev,forecast_dev).numpy()


    
    print(trials)
    print('Dev set accuracy:'+str(acc))
    '''if acc>min_acc:
        ts = time.time()
        model.save('model/model_'+str(acc)+str(trials)+'-801010'+str(ts)+'_20.ckpt')
    '''
    print(forecast_dev)
    print('Validation accuracy: '+str(b(labels_validation,forecast_validation).numpy()))
    
    dev_metrics.append((b(labels_validation,forecast_validation).numpy(),acc, prec, rec))
# sorted(val_acc,key=lambda x:x[0],reverse=True)

sorted(dev_metrics,key=lambda x:x[0],reverse=True)

1
Dev set accuracy:0.39130434
[[0.99208874]
 [0.5247573 ]
 [0.7059119 ]
 [0.02549273]
 [0.00226247]
 [0.566968  ]
 [0.27023652]
 [0.9978361 ]
 [0.9961796 ]
 [0.9980908 ]
 [0.9933648 ]
 [0.7622272 ]
 [0.99488044]
 [0.00191963]
 [0.9980043 ]
 [0.70610064]
 [0.00818041]
 [0.01945496]
 [0.96769965]
 [0.9984404 ]
 [0.99170005]
 [0.09062997]
 [0.9967081 ]]
Validation accuracy: 0.4229249
2
Dev set accuracy:0.3043478
[[0.86031556]
 [0.9949912 ]
 [0.9968997 ]
 [0.00181079]
 [0.00288334]
 [0.9033148 ]
 [0.9967151 ]
 [0.99696815]
 [0.99730945]
 [0.99843544]
 [0.98507476]
 [0.825364  ]
 [0.99722725]
 [0.00435826]
 [0.91671336]
 [0.98445535]
 [0.0429692 ]
 [0.98743933]
 [0.6210517 ]
 [0.99818325]
 [0.06298479]
 [0.9845794 ]
 [0.9981957 ]]
Validation accuracy: 0.3339921
3
Dev set accuracy:0.4347826
[[0.9985777 ]
 [0.9989685 ]
 [0.79621553]
 [0.0015142 ]
 [0.00136882]
 [0.9984069 ]
 [0.0030356 ]
 [0.99894464]
 [0.96980035]
 [0.9989505 ]
 [0.998021  ]
 [0.11405107]
 [0.07601777]
 [0.00125897]
 [0.9984

[(0.50856394, 0.4347826, 0.3846154, 0.5),
 (0.50856394, 0.4347826, 0.4117647, 0.7),
 (0.49143612, 0.5652174, 0.5, 0.7),
 (0.4334651, 0.39130434, 0.375, 0.6),
 (0.41765478, 0.4347826, 0.3846154, 0.5),
 (0.41633728, 0.5217391, 0.47058824, 0.8),
 (0.4044796, 0.3043478, 0.3125, 0.5),
 (0.40316203, 0.39130434, 0.35714287, 0.5),
 (0.40184453, 0.47826087, 0.4375, 0.7),
 (0.3438735, 0.3043478, 0.33333334, 0.6)]

In [64]:
trials=0
min_acc=0.56
val_acc=[]
dev_metrics=[]
while trials<10:
    
    trials+=1
    tf.keras.backend.clear_session()

    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64,
                           kernel_size=9,
                           strides=1,
                           padding="same",
                           activation="tanh",
                           input_shape=windowed_dataset_train.shape[-2:]),
    tf.keras.layers.MaxPooling1D(pool_size=4, strides=1, padding="valid"),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="same", activation="tanh"),
    tf.keras.layers.MaxPooling1D(pool_size=2, strides=1, padding="valid"),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation="tanh"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation="tanh"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    

    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    log_dir = "logs3\\" + datetime.datetime.now().strftime("%d-%m-%Y_%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    history = model.fit(train_set,
                    epochs=100,
                    steps_per_epoch=TRAIN_STEP,
                    validation_data=cross_validation_set,
                    validation_steps=VALIDATION_STEP,
                    verbose=0,
                    callbacks=[tensorboard_callback])
    forecast_train = model.predict(unshuffled_train_set, steps=TRAIN_STEP)
    forecast_validation = model.predict(cross_validation_set, steps=VALIDATION_STEP)
    forecast_dev = model.predict(dev_set, steps=DEV_STEP)
    total_forecast = np.concatenate((forecast_train[:,0], forecast_validation[:,0], forecast_dev[:,0]))
    total_labels = np.concatenate((labels_train, labels_validation, labels_dev))
    b=tf.keras.metrics.BinaryAccuracy(
        name="binary_accuracy", dtype=None,threshold=0.5
    )
    p=tf.keras.metrics.Precision(
        name=None, dtype=None,thresholds=0.5
    )
    
    r=tf.keras.metrics.Recall(
        name=None, dtype=None,thresholds=0.5
    )
    
    
    
#     acc_labels_dev= (forecast_dev < 0.5).astype('int64')

#     precision = precision_score(labels_dev, acc_labels_dev)
#     recall = recall_score(labels_dev, acc_labels_dev)
#     f1 = f1_score(labels_dev, acc_labels_dev)
#     dev_metrics.append((precision, recall, f1, acc))
    
    acc=b(labels_dev,forecast_dev).numpy()
    prec=p(labels_dev,forecast_dev).numpy()
    rec=r(labels_dev,forecast_dev).numpy()


    print(trials)
    print('Dev set accuracy:'+str(acc))
    '''if acc>min_acc:
        ts = time.time()
        model.save('model/model_'+str(acc)+str(trials)+'-801010'+str(ts)+'_20.ckpt')
    ''' 
    print(forecast_dev)
    print('Validation accuracy: '+str(b(labels_validation,forecast_validation).numpy()))
    dev_metrics.append((b(labels_validation,forecast_validation).numpy(),acc, prec, rec))

sorted(dev_metrics,key=lambda x:x[0],reverse=True)

1
Dev set accuracy:0.3043478
[[0.00240639]
 [0.99845314]
 [0.99829197]
 [0.00156829]
 [0.00156656]
 [0.00191045]
 [0.00167638]
 [0.9511432 ]
 [0.9971026 ]
 [0.9985049 ]
 [0.9983014 ]
 [0.9934261 ]
 [0.9984693 ]
 [0.00153637]
 [0.9948213 ]
 [0.99816126]
 [0.39786103]
 [0.9982976 ]
 [0.99732184]
 [0.99713206]
 [0.00450096]
 [0.00811693]
 [0.99844396]]
Validation accuracy: 0.37944663
2
Dev set accuracy:0.4347826
[[0.00271308]
 [0.777292  ]
 [0.0043599 ]
 [0.00174946]
 [0.0015142 ]
 [0.00153732]
 [0.00152758]
 [0.9944469 ]
 [0.01147878]
 [0.99851227]
 [0.99611866]
 [0.02231136]
 [0.99858236]
 [0.00193077]
 [0.02183419]
 [0.99710166]
 [0.00612894]
 [0.99844116]
 [0.9983084 ]
 [0.99858016]
 [0.9950859 ]
 [0.00167599]
 [0.9985718 ]]
Validation accuracy: 0.42193675
3
Dev set accuracy:0.39130434
[[0.0270139 ]
 [0.9968219 ]
 [0.02797008]
 [0.5409967 ]
 [0.00175685]
 [0.00251436]
 [0.00184739]
 [0.96015525]
 [0.00329351]
 [0.99824643]
 [0.28053308]
 [0.9626545 ]
 [0.9983563 ]
 [0.00312725]
 [0.15

[(0.56785244, 0.5217391, 0.46666667, 0.7),
 (0.50988144, 0.3478261, 0.30769232, 0.4),
 (0.50988144, 0.3478261, 0.33333334, 0.5),
 (0.5072464, 0.5217391, 0.45454547, 0.5),
 (0.49407116, 0.39130434, 0.33333334, 0.4),
 (0.49407116, 0.39130434, 0.33333334, 0.4),
 (0.41765478, 0.4347826, 0.36363637, 0.4),
 (0.41765478, 0.4347826, 0.36363637, 0.4),
 (0.4044796, 0.3043478, 0.2857143, 0.4),
 (0.37285903, 0.39130434, 0.33333334, 0.4)]

In [18]:
trials=0
min_acc=0.56
dev_acc=[]
val_acc=[]
dev_metrics=[]
while trials<10:
    
    trials+=1
    tf.keras.backend.clear_session()

    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64,
                           kernel_size=9,
                           strides=1,
                           padding="same",
                           activation="tanh",
                           input_shape=windowed_dataset_train.shape[-2:]),
    tf.keras.layers.MaxPooling1D(pool_size=4, strides=1, padding="valid"),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="same", activation="tanh"),
    tf.keras.layers.MaxPooling1D(pool_size=2, strides=1, padding="valid"),
    tf.keras.layers.GRU(64, return_sequences=True,activation='tanh'),
    tf.keras.layers.GRU(64, return_sequences=True,activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation="tanh"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation="tanh"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    

    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    log_dir = "logs3\\" + datetime.datetime.now().strftime("%d-%m-%Y_%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    history = model.fit(train_set,
                    epochs=100,
                    steps_per_epoch=TRAIN_STEP,
                    validation_data=cross_validation_set,
                    validation_steps=VALIDATION_STEP,
                    verbose=0,
                    callbacks=[tensorboard_callback])
    forecast_train = model.predict(unshuffled_train_set, steps=TRAIN_STEP)
    forecast_validation = model.predict(cross_validation_set, steps=VALIDATION_STEP)
    forecast_dev = model.predict(dev_set, steps=DEV_STEP)
    total_forecast = np.concatenate((forecast_train[:,0], forecast_validation[:,0], forecast_dev[:,0]))
    total_labels = np.concatenate((labels_train, labels_validation, labels_dev))
    
    
#     acc_labels_dev= (forecast_dev < 0.5).astype('int64')

#     precision = precision_score(labels_dev, acc_labels_dev)
#     recall = recall_score(labels_dev, acc_labels_dev)
#     f1 = f1_score(labels_dev, acc_labels_dev)
    
    
    
    b=tf.keras.metrics.BinaryAccuracy(
        name="binary_accuracy", dtype=None,threshold=0.5
    )
    p=tf.keras.metrics.Precision(
        name=None, dtype=None,thresholds=0.5
    )
    
    r=tf.keras.metrics.Recall(
        name=None, dtype=None,thresholds=0.5
    )
    
    acc=b(labels_dev,forecast_dev).numpy()
    prec=p(labels_dev,forecast_dev).numpy()
    rec=r(labels_dev,forecast_dev).numpy()

    print(trials)
    print('Dev set accuracy:'+str(acc))
    '''if acc>min_acc:
        ts = time.time()
        model.save('model/model_'+str(acc)+str(trials)+'-801010'+str(ts)+'_20.ckpt')
    ''' 
    print(forecast_dev)
    dev_metrics.append((b(labels_validation,forecast_validation).numpy(),acc, prec, rec))

    print('Validation accuracy: '+str(b(labels_validation,forecast_validation).numpy()))
# sorted(val_acc,key=lambda x:x[0],reverse=True)

sorted(dev_metrics,key=lambda x:x[0],reverse=True)

1
Dev set accuracy:0.47826087
[[0.00289282]
 [0.9972074 ]
 [0.03516468]
 [0.00318983]
 [0.00286052]
 [0.00285965]
 [0.00288334]
 [0.6814338 ]
 [0.9684676 ]
 [0.99719214]
 [0.9963548 ]
 [0.03526407]
 [0.99723756]
 [0.00288385]
 [0.99326515]
 [0.983613  ]
 [0.02491519]
 [0.9970007 ]
 [0.26507512]
 [0.9858602 ]
 [0.6607301 ]
 [0.00474423]
 [0.99720705]]
Validation accuracy: 0.40184453
2
Dev set accuracy:0.4347826
[[9.8952651e-04]
 [9.3930960e-04]
 [9.6154213e-04]
 [1.0650456e-03]
 [8.9153647e-04]
 [1.4768839e-03]
 [8.9213252e-04]
 [7.7412659e-01]
 [9.9774957e-01]
 [9.9892449e-01]
 [9.9814725e-01]
 [7.5368190e-01]
 [9.9913442e-01]
 [1.0861456e-03]
 [9.9913633e-01]
 [9.9913269e-01]
 [9.9138010e-01]
 [9.9838066e-01]
 [6.0813624e-01]
 [9.8231554e-01]
 [2.8810006e-01]
 [1.1533797e-03]
 [9.9028498e-01]]
Validation accuracy: 0.47826087
3
Dev set accuracy:0.5217391
[[0.01115724]
 [0.12036544]
 [0.9818172 ]
 [0.01684123]
 [0.01257044]
 [0.01265892]
 [0.01326093]
 [0.9860351 ]
 [0.09589958]
 [0.990

[(0.53162056, 0.6086956, 0.54545456, 0.6),
 (0.51185775, 0.47826087, 0.4, 0.4),
 (0.4673913, 0.4347826, 0.3846154, 0.5),
 (0.4673913, 0.4347826, 0.3846154, 0.5),
 (0.4664032, 0.47826087, 0.42857143, 0.6),
 (0.4664032, 0.47826087, 0.4, 0.4),
 (0.465415, 0.5217391, 0.46153846, 0.6),
 (0.465415, 0.5217391, 0.44444445, 0.4),
 (0.4456522, 0.39130434, 0.35714287, 0.5),
 (0.42094862, 0.47826087, 0.41666666, 0.5)]